# Could the SSN residuals have at all resulted from an actual trajectory?

The orbital elements are constructed from Horizons-given position and velocity at end of two-way Doppler from DSS 25 (Goldstone). To avoid hitting Horizons needlessly, the trajectory is constructed using `poliastro`'s orbit propagation simply, without involving atmospheric, solar wind, precession, etc., as all such effects are relatively very small. 3d visualization is constructed for the approach trajectory to help visualize the possibility.

In [1]:
from astropy import units as u
from astropy.time import Time, TimeDelta

from poliastro.util import norm, time_range
from poliastro.frames import Planes
from poliastro.ephem import Ephem
from poliastro.bodies import Sun, Earth

from poliastro.twobody.orbit import Orbit
from poliastro.twobody.sampling import EpochsArray

from astropy.coordinates import solar_system_ephemeris

import numpy as np
import sys
sys.path.append('../')

from sim.stations import dss25, dss34, ssrAltair, ssrMillstone
from sim.tracking import Tracking
from sim.util import describe_orbit, describe_state, describe_trajectory

solar_system_ephemeris.set("de440")
start_epoch = Tracking.NEAR_GOLDSTONE_END.value - 1*u.hour
end_epoch = Tracking.NEAR_CANBERRA_START.value + 1*u.hour
sampling_interval = 60

In [2]:
flyby_epochs = start_epoch + (np.arange(0, (end_epoch-start_epoch)/(1*u.s), sampling_interval) << u.s)
print("Start and end epochs:", start_epoch, flyby_epochs[-1], "[", flyby_epochs.size, "]")

Start and end epochs: 1998-01-23 05:14:55.600 1998-01-23 10:53:55.600 [ 340 ]


In [3]:
near_start_ephem = Ephem.from_horizons("NEAR", start_epoch, attractor=Earth, plane=Planes.EARTH_EQUATOR)
near_start_rv = near_start_ephem.rv(start_epoch)
describe_state(near_start_rv, dss25, start_epoch)

near_orbit = Orbit.from_vectors(Earth, near_start_rv[0], near_start_rv[1], start_epoch)
describe_orbit(near_orbit)


:AT: 1998-01-23 05:14:55.600
Geocentric distance: 65274.0361452193 km speed: 7.6901954431962105 km / s
From Goldstone-25 : range  58960.16319649092 km , range rate  -7.5336544312792695 km / s

::ORBIT::
Plane: Planes.EARTH_EQUATOR
Inclination: 107.97247676097284 deg
Eccentricity: 1.8133809152581064
Semilatus rectum: 19437.797862081028 km
Semimajor axix: -8494.240366015483 km
Periapse radius: 6909.0530033320265 km , altitude: 530.9164033320267 km


In [4]:
near_ephem = near_orbit.to_ephem(EpochsArray(flyby_epochs))
describe_trajectory(near_ephem, dss25)


::TRAJECTORY::
Start and end ranges: 58508.08668617579 km 103599.91462463103 km
Start and end radial speeds: -7.5336544312792615 km / s 7.261684727577173 km / s
Closest: 8845.854508474611 km speed 11.400254809536797 km / s  at  1998-01-23 07:12:55.600
Deflection: 66.08814207688046 deg


Looking down from the spacecraft, Millstone is on the right and Altair on the left. Earth's rotation brings Altair closer and Millstone farther, which explains the steeper slope of the Altair residuals.

The linearity and exact relation to range would be satisfied by a very slightly higher, almost parallel trajectory, since the range data differences are below 1 km at almost 35,000 km at the start of the SSN tracks, and 450 m at 15,000 km at the end. 

In [5]:
from czml3.widget import CZMLWidget
from poliastro.czml.extract_czml import CZMLExtractor

czml = CZMLExtractor(flyby_epochs[0], flyby_epochs[-1], flyby_epochs.size)
#dss25.add_to_czml(czml, "#333")
#dss34.add_to_czml(czml, "#333")
ssrAltair.add_to_czml(czml, "#111")
ssrMillstone.add_to_czml(czml, "#111")
czml.add_orbit(near_orbit)

CZMLWidget(czml.get_document())

CZMLWidget(document=Document(_values=[Preamble(id='document', version='1.0', name='document_packet', description=None, clock=IntervalValue(_start=<Time object: scale='tdb' format='isot' value=1998-01-23T05:14:55.600>, _end=<Time object: scale='tdb' format='isot' value=1998-01-23T10:53:55.600>, _value=Clock(currentTime=datetime.datetime(1998, 1, 23, 5, 14, 55, 600000, tzinfo=datetime.timezone.utc), multiplier=60, range=<ClockRanges.LOOP_STOP: 3>, step=<ClockSteps.SYSTEM_CLOCK_MULTIPLIER: 2>))), Packet(id='custom_properties', delete=None, name=None, parent=None, description=None, availability=None, properties={'custom_attractor': True, 'ellipsoid': [{'array': (6378136.6, 6378136.6, 6356751.9)}], 'map_url': ('https://upload.wikimedia.org/wikipedia/commons/c/c4/Earthmap1000x500compac.jpg',), 'scene3D': True}, position=None, orientation=None, viewFrom=None, billboard=None, box=None, corridor=None, cylinder=None, ellipse=None, ellipsoid=None, label=None, model=None, path=None, point=None, polygon=None, polyline=None, rectangle=None, tileset=None, wall=None), Packet(id='GS0', delete=None, name=None, parent=None, description='Altair', availability=TimeInterval(_start=<Time object: scale='tdb' format='isot' value=1998-01-23T05:14:55.600>, _end=<Time object: scale='tdb' format='isot' value=1998-01-23T10:53:55.600>), properties=None, position=Position(epoch=None, interpolationAlgorithm=None, interpolationDegree=None, delete=None, referenceFrame=None, cartesian=[-6143471.429543289, 1364300.4818256705, 1034339.9636133286], cartographicRadians=None, cartographicDegrees=None, cartesianVelocity=None, reference=None), orientation=None, viewFrom=None, billboard=Billboard(horizontalOrigin=None, verticalOrigin=None, image='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAYAAAAf8/9hAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAAACvSURBVDhPrZDRDcMgDAU9GqN0lIzijw6SUbJJygUeNQgSqepJTyHG91LVVpwDdfxM3T9TSl1EXZvDwii471fivK73cBFFQNTT/d2KoGpfGOpSIkhUpgUMxq9DFEsWv4IXhlyCnhBFnZcFEEuYqbiUlNwWgMTdrZ3JbQFoEVG53rd8ztG9aPJMnBUQf/VFraBJeWnLS0RfjbKyLJA8FkT5seDYS1Qwyv8t0B/5C2ZmH2/eTGNNBgMmAAAAAElFTkSuQmCC', show=True, scale=None, eyeOffset=None), box=None, corridor=None, cylinder=None, ellipse=None, ellipsoid=None, label=Label(horizontalOrigin=None, verticalOrigin=None, show=True, text=None, font='11pt Lucida Console', style=<LabelStyles.FILL: 1>, scale=None, showBackground=None, backgroundColor=None, fillColor=Color(epoch=None, interpolationAlgorithm=None, interpolationDegree=None, delete=None, rgba='#111', rgbaf=None), outlineColor=None, outlineWidth=1.0, pixelOffset=None), model=None, path=None, point=None, polygon=None, polyline=None, rectangle=None, tileset=None, wall=None), Packet(id='GS1', delete=None, name=None, parent=None, description='Millstone', availability=TimeInterval(_start=<Time object: scale='tdb' format='isot' value=1998-01-23T05:14:55.600>, _end=<Time object: scale='tdb' format='isot' value=1998-01-23T10:53:55.600>), properties=None, position=Position(epoch=None, interpolationAlgorithm=None, interpolationDegree=None, delete=None, referenceFrame=None, cartesian=[1492298.8700490487, -4457675.543085808, 4296323.883602885], cartographicRadians=None, cartographicDegrees=None, cartesianVelocity=None, reference=None), orientation=None, viewFrom=None, billboard=Billboard(horizontalOrigin=None, verticalOrigin=None, image='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAYAAAAf8/9hAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAAACvSURBVDhPrZDRDcMgDAU9GqN0lIzijw6SUbJJygUeNQgSqepJTyHG91LVVpwDdfxM3T9TSl1EXZvDwii471fivK73cBFFQNTT/d2KoGpfGOpSIkhUpgUMxq9DFEsWv4IXhlyCnhBFnZcFEEuYqbiUlNwWgMTdrZ3JbQFoEVG53rd8ztG9aPJMnBUQf/VFraBJeWnLS0RfjbKyLJA8FkT5seDYS1Qwyv8t0B/5C2ZmH2/eTGNNBgMmAAAAAElFTkSuQmCC', show=True, scale=None, eyeOffset=None), box=None, corridor=None, cylinder=None, ellipse=None, ellipsoid=None, label=Label(horizontalOrigin=None, verticalOrigin=None, show=True, text=None, font='11pt Lucida Console',